<a href="https://colab.research.google.com/github/DavidTorres-sys/proyecto-modelos-y-simulacion-udea/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports Necesarios


In [58]:
# Librerías principales
import pandas as pd
import numpy as np
import os

# Preprocesamiento y modelado
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report

# Balanceo de clases
from imblearn.over_sampling import SMOTE

# Modelo de Gradient Boosting
from lightgbm import LGBMClassifier

In [59]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [60]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.15GB/s]


In [61]:
!unzip udea*.zip > /dev/null
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [62]:
def read_file(file):
    try:
        return pd.read_csv(file)
    except FileNotFoundError:
        print(f"Error: El archivo '{file}' no se encontró.")
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo: {e}")


In [63]:
df = read_file("train.csv")
print("Tamaño del dataset:", df.shape)
df.head()

Tamaño del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# 1. Cargamos los datos



In [64]:
# Cargamos los archivos de entrenamiento y prueba
df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Mostramos las primeras filas para inspección rápida
df.head()


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# 2. LIMPIEZA + DETECCIÓN NUMÉRICA/CATEGÓRICA

In [65]:
# Función para detectar columnas numéricas y categóricas
def detect_numeric(df):
    num_cols, cat_cols = [], []
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors="raise")
            num_cols.append(col)
        except:
            cat_cols.append(col)
    return num_cols, cat_cols

In [66]:
# Detectamos columnas en train y test
train_num_cols, train_cat_cols = detect_numeric(df)
test_num_cols, test_cat_cols = detect_numeric(test_df)

# Rellenamos valores faltantes
df[train_num_cols] = df[train_num_cols].fillna(df[train_num_cols].median())
df[train_cat_cols] = df[train_cat_cols].fillna("DESCONOCIDO")

test_df[test_num_cols] = test_df[test_num_cols].fillna(test_df[test_num_cols].median())
test_df[test_cat_cols] = test_df[test_cat_cols].fillna("DESCONOCIDO")

# Eliminamos columnas duplicadas si existen
df = df.loc[:, ~df.columns.duplicated()]
test_df = test_df.loc[:, ~test_df.columns.duplicated()]


# 3. Definición de Target

In [67]:
# Verificamos si ya existe la columna 'RENDIMIENTO_GLOBAL'
if "RENDIMIENTO_GLOBAL" in df.columns:
    y = df["RENDIMIENTO_GLOBAL"]
else:
    # Calculamos promedio de indicadores como proxy de desempeño
    df["DESEMPENO"] = df[["INDICADOR_1", "INDICADOR_2", "INDICADOR_3", "INDICADOR_4"]].mean(axis=1)
    # Cuartiles para generar etiquetas multi-clase
    df["RENDIMIENTO_GLOBAL"] = pd.qcut(
        df["DESEMPENO"], q=4,
        labels=["bajo", "medio-bajo", "medio-alto", "alto"]
    )
    y = df["RENDIMIENTO_GLOBAL"]

# Separamos features, eliminando target y columna ID si existen
X = df.drop(columns=["RENDIMIENTO_GLOBAL", "ID"], errors="ignore")


# 4. OrdinalEncoder (seguro para categorías no visibles)

In [68]:
# Detectamos columnas categóricas
cat_cols = X.select_dtypes(include=["object"]).columns

# Inicializamos OrdinalEncoder con opción de manejar categorías desconocidas
oe = OrdinalEncoder(
    handle_unknown="use_encoded_value",
    unknown_value=-1
)

# Transformamos las columnas categóricas del entrenamiento
X[cat_cols] = oe.fit_transform(X[cat_cols])

# Preparamos test_features: eliminamos ID y alineamos columnas
test_features = test_df.drop(columns=["ID"], errors="ignore")
for col in cat_cols:
    if col not in test_features.columns:
        test_features[col] = "DESCONOCIDO"

# Aseguramos mismo orden de columnas y transformamos categóricas
test_features = test_features[X.columns]
test_features[cat_cols] = oe.transform(test_features[cat_cols])

# 5. TRAIN/VALIDATION SPLIT

In [69]:
# Dividimos los datos en entrenamiento y validación
# Estratificamos para mantener proporción de clases
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


# 6. SMOTE (balanceo de clases)

In [70]:
# Aplicamos SMOTE para generar más ejemplos de clases minoritarias
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)


# 7. LIGHTGBM MODEL

In [71]:
# Inicializamos el modelo LightGBM para clasificación multi-clase
model = LGBMClassifier(
    n_estimators=600,
    learning_rate=0.03,
    num_leaves=64,
    max_depth=-1,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    min_child_samples=40,
    lambda_l1=1,
    lambda_l2=1,
    objective="multiclass",
    random_state=42
)

# Entrenamos el modelo con los datos balanceados
model.fit(X_train_resampled, y_train_resampled)


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] 

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, feature_fraction=0.8,
               lambda_l1=1, lambda_l2=1, learning_rate=0.03,
               min_child_samples=40, n_estimators=600, num_leaves=64,
               objective='multiclass', random_state=42)

# 8. VALIDATION

In [72]:
# Evaluamos desempeño sobre el set de validación
pred_val = model.predict(X_val)
print("===== VALIDATION REPORT =====")
print(classification_report(y_val, pred_val))


[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
===== VALIDATION REPORT =====
              precision    recall  f1-score   support

        alto       0.55      0.64      0.59     43905
        bajo       0.47      0.57      0.51     43246
  medio-alto       0.33      0.27      0.29     42905
  medio-bajo       0.33      0.27      0.30     43069

    accuracy                           0.44    173125
   macro avg       0.42      0.44      0.42    173125
weighted avg       0.42      0.44      0.4

# 9. PREDICT TEST SET

In [73]:
# Generamos predicciones para el set de prueba
test_predictions = model.predict(test_features)

[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


# 10. BUILD SUBMISSION CSV

In [74]:
# Construimos el DataFrame final de submission
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "RENDIMIENTO_GLOBAL": test_predictions
})

# Guardamos en CSV listo para Kaggle
submission.to_csv("submission.csv", index=False)

print("\nSUCCESS: submission.csv created!")


SUCCESS: submission.csv created!


In [75]:
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f submission.csv -m "raul ramos submission with linear model"

100% 4.05M/4.05M [00:00<00:00, 17.4MB/s]
400 Client Error: Bad Request for url: https://www.kaggle.com/api/v1/competitions/submissions/submit/udea-ai-4-eng-20252-pruebas-saber-pro-colombia
